In [1]:
import pandas as pd
import numpy as np

# For reproducibility
seed = 27
np.random.seed(seed)

train = pd.read_csv("modified_data/train_expanded.csv")
validate = pd.read_csv("modified_data/validate.csv")

# Split into input (X) and output (Y) variables
x_train = train.values[:,2:]
y_train = train.values[:,1]

x_validate = validate.values[:,2:]
y_validate = validate.values[:,1]

In [2]:
'''Trains a simple convnet on the MNIST dataset.

Gets to 99.25% test accuracy after 12 epochs
(there is still a lot of margin for parameter tuning).
16 seconds per epoch on a GRID K520 GPU.
'''

from __future__ import print_function
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.callbacks import ReduceLROnPlateau
from keras import backend as K

batch_size = 512
num_classes = 10
epochs = 20

# input image dimensions
img_rows, img_cols = 28, 28

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_validate = x_validate.reshape(x_validate.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_validate = x_validate.reshape(x_validate.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_validate = x_validate.astype('float32')
x_train /= 255
x_validate /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_validate.shape[0], 'validation samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_validate = keras.utils.to_categorical(y_validate, num_classes)


model = Sequential()
model.add(Conv2D(32, kernel_size=(5, 5),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
          
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
          
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
 
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor = 0.5,
                              patience = 2, min_lr=0.0001)

model.fit(x_train, y_train,
          batch_size = batch_size,
          epochs = epochs,
          verbose = 1,
          validation_data = (x_validate, y_validate),
          callbacks = [reduce_lr])

score = model.evaluate(x_validate, y_validate, verbose = 0)
print('Validation loss:', score[0])
print('Validation accuracy:', score[1])

Using TensorFlow backend.
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


x_train shape: (264600, 28, 28, 1)
264600 train samples
4200 validation samples
Train on 264600 samples, validate on 4200 samples
Epoch 1/20
264600/264600 [==============================] - 60s 226us/step - loss: 0.3326 - acc: 0.8906 - val_loss: 0.0343 - val_acc: 0.9888
Epoch 2/20
264600/264600 [==============================] - 58s 218us/step - loss: 0.0734 - acc: 0.9774 - val_loss: 0.0227 - val_acc: 0.9914
Epoch 3/20
264600/264600 [==============================] - 58s 218us/step - loss: 0.0506 - acc: 0.9842 - val_loss: 0.0212 - val_acc: 0.9936
Epoch 4/20
264600/264600 [==============================] - 58s 218us/step - loss: 0.0399 - acc: 0.9877 - val_loss: 0.0183 - val_acc: 0.9940
Epoch 5/20
264600/264600 [==============================] - 58s 218us/step - loss: 0.0337 - acc: 0.9894 - val_loss: 0.0149 - val_acc: 0.9945
Epoch 6/20
264600/264600 [==============================] - 58s 218us/step - loss: 0.0290 - acc: 0.9906 - val_loss: 0.0132 - val_acc: 0.9952
Epoch 7/20
264600/264600

In [3]:
test = pd.read_csv("raw_data/test.csv").values[:,:]

if K.image_data_format() == 'channels_first':
    test = test.reshape(test.shape[0], 1, img_rows, img_cols)
else:
    test = test.reshape(test.shape[0], img_rows, img_cols, 1)

test = test.astype('float32')
test /= 255

pred = np.argmax(model.predict(test), axis = 1)
submission = pd.DataFrame(data = pred, columns = ['Label'])
submission['ImageId'] = submission.index
submission = submission[['ImageId', 'Label']]

#submission.to_csv("submissions/Dec-31-2017.csv", index = False)